In [ ]:
import datetime
import time
import requests

import json
import os
import pandas as pd
import numpy as np

#threading
import threading
import timeit

from email.mime.text import MIMEText
import smtplib

import gc
import sys

#declare global lock object
global lock
lock = threading.Lock()

In [ ]:
def symbol_downloader(symbol, directory, days=600, days_ago=0):
    dt = datetime.datetime.now()
    UnixTime = int(time.mktime(dt.timetuple()))
    #web variables
    url = 'https://query1.finance.yahoo.com/v8/finance/chart/'+symbol+'?period1='+str(UnixTime-86400*(days+days_ago))+'&period2='+str(UnixTime-86400*days_ago)+'&interval=1d&indicators=quote%7Csma~60&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-CA&region=CA&corsDomain=ca.finance.yahoo.com'
    #proxies
    http_proxy  = ''
    https_proxy = ''
    ftp_proxy   = ''
    proxyDict = { 
                  "http"  : http_proxy, 
                  "https" : https_proxy, 
                  "ftp"   : ftp_proxy
                }

    hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
       'Connection': 'keep-alive'}
    
    try:
        resp = requests.get(url, headers=hdr)
        if resp.status_code != 200:
            # This means something went wrong.
            print(resp.status_code)
        data = json.loads(resp.text)
        #json unpack
        timestamp = data['chart']['result'][0]['timestamp']
        timestamp = [datetime.datetime.fromtimestamp(x).strftime('%Y%m%d') for x in timestamp]
        quote = data['chart']['result'][0]['indicators']['quote'][0]
        sma60 = data['chart']['result'][0]['indicators']['sma'][0]['sma']
        stock_df = pd.DataFrame(quote)
        #index is symbol and timestamp
        stock_df.index = [str(x) for x in timestamp]

        #moving averages
        stock_df['sma60'] = sma60
        stock_df['vol20'] = stock_df['volume'].rolling(window=20).mean()
        stock_df['sma5'] = stock_df['close'].rolling(window=5).mean()
        stock_df['sma8'] = stock_df['close'].rolling(window=8).mean()
        stock_df['sma13'] = stock_df['close'].rolling(window=13).mean()
        stock_df['sma21'] = stock_df['close'].rolling(window=21).mean()
        stock_df['sma34'] = stock_df['close'].rolling(window=34).mean()
        stock_df['sma55'] = stock_df['close'].rolling(window=55).mean()
        stock_df['sma89'] = stock_df['close'].rolling(window=89).mean()
        stock_df['sma144'] = stock_df['close'].rolling(window=144).mean()
        stock_df['sma233'] = stock_df['close'].rolling(window=233).mean()
        stock_df = stock_df.dropna()

        #stock_df.drop(['close', 'high', 'low', 'open','volume','sma','vol20'], axis=1, inplace=True)
        stock_df = stock_df.dropna()
        stock_df = stock_df[~(stock_df == np.inf).any(axis=1)]
        if len(stock_df) != 0:
            stock_df.to_csv('{}{}.csv'.format(directory,symbol[:4]))
        try:
            lock.acquire()
            #clear memory
            gc.collect()
        finally:
            lock.release()
        
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        raise SystemExit(e)

In [ ]:
if __name__ == '__main__':

    #start timer
    start = timeit.default_timer()

    symbolslist = open('symbols.txt').read().split('\n')

    directory = './data_nasdaq/'

    threadlist = []

    for u in symbolslist:

        t = threading.Thread(target = symbol_downloader,args=(u,directory))
        t.start()
        threadlist.append(t)
        #sets top limit of active threads to 20
        while threading.activeCount()>20:
            a=0
        #print threading.activeCount()

    for b in threadlist:
        b.join()
    print ("# of threads: ", len(threadlist))


In [ ]:
#Uncle's pattern
import traceback
gc.collect()
to_send = ''
directory = './data_nasdaq/'
# get list with filenames in folder and throw away all non ncsv
files = [f for f in os.listdir(directory) if f.endswith('.csv')]
for f in files:
    try:
        ticker_df = pd.read_csv(os.path.join(directory, f))
        jones_df = pd.read_csv(os.path.join(directory,'^DJI.csv'))
        
        stock_df = pd.merge(ticker_df, jones_df, left_index=True, right_index=True)

        for i in range(int((len(stock_df)*0.1))): #engulfing candel pattern looping through all available data 
            
            window = 59 #number of days back from today to look at for slope
            
            i += 1 #start range from 1 instead of 0
            
            openTday =  float(stock_df['open_x'].iloc[-i])
            closeTday =  float(stock_df['close_x'].iloc[-i])
            openYday = float(stock_df['open_x'].iloc[-(i+1)])
            closeYday = float(stock_df['close_x'].iloc[-(i+1)])
            
            sma5 = float(stock_df['sma5_x'].iloc[-i])
            sma8 = float(stock_df['sma8_x'].iloc[-i])
            sma13 = float(stock_df['sma13_x'].iloc[-i])
            sma21 = float(stock_df['sma21_x'].iloc[-i])
            
            sma34 = float(stock_df['sma34_x'].iloc[-i])
            
            sma55 = float(stock_df['sma55_x'].iloc[-i])
            sma89 = float(stock_df['sma89_x'].iloc[-i])
            sma144 = float(stock_df['sma144_x'].iloc[-i])
            
            sma233 = float(stock_df['sma233_x'].iloc[-i])
            
            mktVlcty = float(stock_df['volume_x'].iloc[-i])*float(stock_df['close_x'].iloc[-i])
            
            volume = (float(stock_df['volume_x'].iloc[-i])+0.001)/(float(stock_df['vol20_x'].iloc[-i])+0.001)
            
            j_open0 =  float(stock_df['open_y'].iloc[-i])
            j_close0 =  float(stock_df['close_y'].iloc[-i])
            
            stockPChange = (closeTday-openTday)/openTday 
            jonesPChange = (j_close0-j_open0)/j_open0

            #core logic
            if  stockPChange > abs(jonesPChange)*5\
                and openTday < closeTday \
                and sma5<closeTday \
                and sma5 > sma8 and sma8 > sma13 \
                and sma89 < sma5 and sma55 < sma5 \
                and closeTday > sma89 \
                and sma5 < sma144 and sma34 < sma144 and sma55 < sma144 and sma89<sma144\
                and sma144 < sma233\
                and closeTday <= 10 and closeTday >= 0.5 \
                and mktVlcty > 1000000\
                and volume >= 2:

                to_send += '{} has uncle"s pattern with high volume on {}, and is under $10 \n'.format(f[:-4],(stock_df['Unnamed: 0_x'].iloc[-i]))
                print ('{} has uncle"s pattern with high volume on {}, and is under $10 \n'.format(f[:-4],(stock_df['Unnamed: 0_x'].iloc[-i])))
    except IndexError:
        print("{} has too few rows".format(f))
        pass
    except Exception:
        print(f,stock_df['Unnamed: 0_x'].iloc[-i])
        print(traceback.format_exc())
        # or
        print(sys.exc_info()[2])
        pass

In [ ]:
print (to_send)

In [ ]:
#Ascending Triangle pattern
import traceback
#clear memory
gc.collect()
to_send = ''
directory = './data_nasdaq/'
# get list with filenames in folder and throw away all non ncsv
files = [f for f in os.listdir(directory) if f.endswith('.csv')]
for f in files:
    try:
        stock_df = pd.read_csv(os.path.join(directory, f))

        for i in range(int((len(stock_df)*0.5))): #looping through last % of data
            
            window = 29 #number of days back from today to look at
            
            i += 1 #start range from 1 instead of 0
            
            openTday =  float(stock_df['open'].iloc[-i])
            closeTday =  float(stock_df['close'].iloc[-i])
            volumeTdayP = (float(stock_df['volume'].iloc[-i])+0.001)/(float(stock_df['vol20'].iloc[-i])+0.001)

            sma5 = float(stock_df['sma5'].iloc[-(i+1)])
            sma8 = float(stock_df['sma8'].iloc[-(i+1)])
            sma13 = float(stock_df['sma13'].iloc[-(i+1)])
            
            #normalize with moving average
            stock_df['high'] = stock_df['high']/stock_df['sma5']
            stock_df['low'] = stock_df['low']/stock_df['sma5']
            stock_df['volume'] = stock_df['volume']/stock_df['vol20']
            
            #range of variables
            df_normalized_high = stock_df['high'][-(i+window):-(i+1)]
            df_normalized_low = stock_df['low'][-(i+window):-(i+1)]
            df_normalized_vol = stock_df['volume'].iloc[-(i+window):-(i+1)]


            #top x points of high
            df_normalized_high=df_normalized_high.nlargest(3,keep='first')
            high_index_diff1 = abs(df_normalized_high.sort_values().index[0] -df_normalized_high.sort_values().index[1])
            high_index_diff2 = abs(df_normalized_high.sort_values().index[1] -df_normalized_high.sort_values().index[2])
            #lowest x points of low
            df_normalized_low=df_normalized_low.nsmallest(3,keep='first')
            low_index_diff1 = abs(df_normalized_low.sort_values().index[0] -df_normalized_low.sort_values().index[1])
            low_index_diff2 = abs(df_normalized_low.sort_values().index[1] -df_normalized_low.sort_values().index[2])
            #compute slopes
            slope_high_Xd = np.polyfit(df_normalized_high.index, df_normalized_high, 1)[0]
            slope_low_Xd = np.polyfit(df_normalized_low.index, df_normalized_low, 1)[0]
            slope_volume_Xd = np.polyfit(df_normalized_vol.index, df_normalized_vol, 1)[0]
            
            #print(f,slope_high_Xd,slope_low_Xd,slope_volume_Xd)
                      
            #core logic
            if closeTday > openTday\
                and high_index_diff1 > 3\
                and high_index_diff2 > 3\
                and low_index_diff1 > 3\
                and low_index_diff2 > 3\
                and slope_high_Xd > -0.01\
                and slope_low_Xd > (slope_high_Xd+0.0001)\
                and sma5<closeTday\
                and sma8<closeTday\
                and sma13<closeTday\
                and closeTday <= 50 and closeTday >= 0.5\
                and volumeTdayP >3:

                to_send += '{} has Ascending Triangle pattern on {}, and is under $20 \n'.format(f[:-4],(stock_df['Unnamed: 0'].iloc[-i]))
                print ('{} has Ascending Triangle pattern on {}, and is under $20 \n'.format(f[:-4],(stock_df['Unnamed: 0'].iloc[-i])))
        
    except IndexError:
        pass
    except Exception:
        print(f,stock_df['Unnamed: 0'].iloc[-i])
        print(traceback.format_exc())
        # or
        print(sys.exc_info()[2])

In [ ]:
#email output
with open('C:\\Users\\Richard\\Desktop\\Python\\hotmail.txt', 'rb') as f:
    email_list = str(f.read()).split(',')
    emailAddress = email_list[0][2:]
    password = email_list[1][:-1]

    msg = MIMEText(to_send)
    recipients = [emailAddress, 'michelleusdenski@gmail.com','Guowei88888@msn.com']#'jacob.si@outlook.com','Greggh_101@hotmail.com','Guowei88888@msn.com']
    msg['Subject'] = '%s stock analysis: Daily Candle potentials - %s' % (str(datetime.datetime.today().strftime('%Y-%m-%d')), str(len(to_send.split('\n'))-1))
    msg['From'] = emailAddress
    msg['To'] = ', '.join(recipients)
    try:
        s = smtplib.SMTP('smtp-mail.outlook.com', 25)
        s.ehlo()  # Hostname to send for this command defaults to the fully qualified domain name of the local host.
        s.starttls()  # Puts connection to SMTP server in TLS mode
        s.ehlo()
        s.login(emailAddress, password)
        s.sendmail(emailAddress, recipients, msg.as_string())
        s.quit()
        print ('email sent to: %s' % emailAddress)
    except:
        raise